In [1]:
!pip install praw python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 1.0 MB/s eta 0:00:00


In [2]:
import praw

client_id = "XJksspIo9onybjJ_oNXTDA"
client_secret = "Fhl...LaAlXw"#my client secret of reddit
user_agent = "PersonaApp/0.1 by kinngofworld"

# Creating Reddit instance
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)


In [3]:
def fetch_user_data(username, limit=100):
    user = reddit.redditor(username)
    posts = []
    comments = []

    try:
        for post in user.submissions.new(limit=limit):
            post_text = f"Title: {post.title}\nBody: {post.selftext}\nURL: {post.url}\n"
            posts.append(post_text)
    except Exception as e:
        print("Error fetching posts:", e)

    try:
        for comment in user.comments.new(limit=limit):
            comment_text = f"Comment: {comment.body}\nSubreddit: {comment.subreddit}\n"
            comments.append(comment_text)
    except Exception as e:
        print("Error fetching comments:", e)

    return posts, comments


In [5]:
username = "kojied"  # Example user
posts, comments = fetch_user_data(username)

print(f"Total Posts: {len(posts)}")
print(f"Total Comments: {len(comments)}")

print("\nSample Post:\n", posts[0][:500])
print("\nSample Comment:\n", comments[0][:500])


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Total Posts: 31
Total Comments: 100

Sample Post:
 Title: I feel violated by intern season
Body: There's this bar that I frequent a few blocks away from my house. Technically different neighborhood, but generally has a mature vibe with people who like music and to dance. Today when I went to the bar and all of a sudden I felt like I was at a college basement party. I thought that I must have been hallucinating, but Then a girl pointed at me and referred to me as the old person. That's when I knew I had to get out of there.
 I was at the wrong pa

Sample Comment:
 Comment: Sorry to hear that man, in the future you may want to have multiple granaries, one close to your berries/deer/fish and one for your veggies/apples/honey.
Also I just store ale at the tavern or keep it in the brewery for this reason
Subreddit: ManorLords



In [6]:
all_text = "\n\n".join(posts + comments)  # posts/comments fetched

In [7]:
def build_persona_prompt(username, user_text):
    return f"""
You are an AI profiler. Based on the following Reddit user's posts and comments, generate a detailed user persona.

**Persona should include**:
- Age group
- Likely gender
- Occupation or field
- Hobbies/interests
- Writing style or tone
- Subreddits they frequent
- Possible location or background
- Notable beliefs or opinions

**Important**: After each trait, cite the exact post or comment (or quote a line) that helped you decide it.

### Reddit User: {username}
### Posts & Comments:
{user_text}
"""

In [10]:
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="sk-or-v1-499e....c5372a26c"  #MY DEEPSEEK R1 API KEY
)

def generate_persona_openrouter(prompt):
    completion = client.chat.completions.create(
        model="deepseek/deepseek-r1:free",
        messages=[
            {"role": "user", "content": prompt}
        ],
        extra_headers={
            "HTTP-Referer": "https://github.com/Abhiudai12",
        }
    )
    return completion.choices[0].message.content


In [13]:
username = "Hungry-Move-6603"
user_text = "\n\n".join(posts + comments)
prompt = build_persona_prompt(username, user_text)
persona = generate_persona_openrouter(prompt)

print(persona)


### User Persona for Hungry-Move-6603  

**Age Group**: **Late 20s to mid-30s**  
**Supporting Evidence**:  
> "I’ve only been here [NYC] for three years as well." (r/newyorkcity post)  
> Referred to as "the old person" by college-aged bar-goers, suggesting they are older than early 20s but still young enough to frequent nightlife.  

---  

**Likely Gender**: **Male**  
**Supporting Evidence**:  
> Gave detailed advice on improving "conventional attractiveness" to a male user in r/GenZ (e.g., fitness, shaving, lasik).  
> Engaged in r/seduction with advice focused on approaching women.  

---  

**Occupation or Field**: **Tech Professional (iOS/visionOS Developer)**  
**Supporting Evidence**:  
> "I’m an iOS developer building in visionOS." (r/visionosdev post)  
> Posts about Vision Pro workflows, GitHub, terminal limitations, and Xcode (r/VisionPro).  
> Interest in spatial computing and AI integration (e.g., ChatGPT in Vision Pro).  

---  

**Hobbies/Interests**:  
- **Gaming**: 

In [15]:
with open("Hungry-Move-6603.txt", "w", encoding="utf-8") as f:
    f.write(persona)